In [ ]:
#%matplotlib inline
%matplotlib notebook

from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
from six.moves import cPickle as pickle
from sklearn.manifold import TSNE
from collections import Counter
from glob import glob

plt.style.use('ggplot')

### Helper Functions

In [ ]:
def gc(sequence):
    sequence = sequence.upper()
    return (sequence.count('G') + sequence.count('C')) / float(len(sequence))


def seq_entropy(sequence):
    bases = {'A', 'C', 'G', 'T'}
    c = Counter(sequence.upper())
    tot = float(sum([c[b] for b in bases]))
    c = {b: c[b]/tot for b in bases}
    
    return sum([(-c[b]) * np.log2(c[b]) for b in bases if c[b] > 0])


def patten2number(sequence):
    """ Converts DNA sequence into an int. """
    BASE_TO_NUMBER = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    if len(sequence) == 0:
        return 0
    last_base = sequence[-1]
    prefix = sequence[:-1]
    return 4 * patten2number(prefix) + BASE_TO_NUMBER[last_base]
    

def number2patten(number, kmer_size):
    """ Converts an int into a string with k nucleotides. """
    NUMBER_TO_BASE = ('A', 'C', 'G', 'T')
    if kmer_size == 1:
        return NUMBER_TO_BASE[number]
    prefix_index = number // 4
    base = NUMBER_TO_BASE[number % 4]
    return number2patten(prefix_index, kmer_size - 1) + base

### Load Embeddings

In [ ]:
kmer_sizes = np.array([3, 4, 5])
index_offset = np.concatenate(([0], (4**kmer_sizes))).cumsum()

pickle_file = '../max5_min3_mers_10padding_64embedding_epoch1_batch0.pickle'
with open(pickle_file, 'rb') as f:
    pre_normalized_embeddings = pickle.load(f)

glob_str = '../max5_min3_mers_10padding_64embedding_epoch1_batch*.pickle'
emb_files = {int(f.rstrip('.pickle').split('batch')[1]): f for f in glob(glob_str)}
pickle_file = emb_files[sorted(emb_files)[-1]]
print(pickle_file)
        
with open(pickle_file, 'rb') as f:
    post_normalized_embeddings = pickle.load(f)

print(pre_normalized_embeddings.shape)
print(post_normalized_embeddings.shape)

# Get 5-mers
pre_normalized_embeddings = pre_normalized_embeddings[index_offset[2]:index_offset[3]]
post_normalized_embeddings = post_normalized_embeddings[index_offset[2]:index_offset[3]]
print('Only 5mers:', pre_normalized_embeddings.shape)
print('Only 5mers:', post_normalized_embeddings.shape)

### Convert into 2D Space (t-distributed stochastic neighbor embedding (t-SNE))

In [ ]:
KMER_SIZE = 5
#NUM_POINTS = 5000

#random_kmers = np.random.choice(4**KMER_SIZE, NUM_POINTS, replace=False)
#random_kmers.sort()
#print(len(random_kmers), random_kmers[:10])

In [ ]:
# Subset
#labels = random_kmers  # Needs to be sorted

# All
labels = range(4**KMER_SIZE)

labels_entropy = [seq_entropy(number2patten(s, KMER_SIZE)) for s in labels]
labels_gc = [gc(number2patten(s, KMER_SIZE)) for s in labels]

# Frequence in hg38
labels_frequence = {}
for kmer in open('../reference_vocabulary/all_{}-mers.tsv'.format(KMER_SIZE)):
    seq, freq = kmer.split()
    seq = patten2number(seq)
    freq = int(freq)
    if seq in labels:
        labels_frequence[seq] = freq
    
labels_frequence = [np.log10(labels_frequence[k]) for k in sorted(labels_frequence.keys())]

# Frequence in hg38 LINE
labels_frequence_line = {}
for kmer in open('../reference_vocabulary/rmsk_LINE_{}-mers.tsv'.format(KMER_SIZE)):
    seq, freq = kmer.split()
    seq = patten2number(seq)
    freq = int(freq)
    if seq in labels:
        labels_frequence_line[seq] = freq

labels_frequence_line = [np.log10(labels_frequence_line[k]) for k in sorted(labels_frequence_line.keys())]
    
# Frequence in hg38 SINE
labels_frequence_sine = {}
for kmer in open('../reference_vocabulary/rmsk_SINE_{}-mers.tsv'.format(KMER_SIZE)):
    seq, freq = kmer.split()
    seq = patten2number(seq)
    freq = int(freq)
    if seq in labels:
        labels_frequence_sine[seq] = freq

labels_frequence_sine = [np.log10(labels_frequence_sine[k]) for k in sorted(labels_frequence_sine.keys())]

    
print(len(labels_entropy), labels_entropy[:10])
print(len(labels_gc), labels_gc[:10])
print(len(labels_frequence), labels_frequence[:10])
print(len(labels_frequence_line), labels_frequence_line[:10])
print(len(labels_frequence_sine), labels_frequence_sine[:10])

In [ ]:
tsne = TSNE(perplexity=50, n_components=2, init='pca', n_iter=10000)

## Subset
#pre_two_d_embeddings = tsne.fit_transform([pre_normalized_embeddings[s, :] for s in random_kmers])
#post_two_d_embeddings = tsne.fit_transform([post_normalized_embeddings[s, :] for s in random_kmers])

## All
pre_two_d_embeddings = tsne.fit_transform(pre_normalized_embeddings)
post_two_d_embeddings = tsne.fit_transform(post_normalized_embeddings)

### Looking at the Result

In [ ]:
def plot(x, y, labels, title):
    norm = colors.Normalize(vmin=min(labels), vmax=max(labels))
    plt.scatter(x, y, c=labels, alpha=0.3, cmap=plt.get_cmap('jet'), norm=norm)
    plt.colorbar()
    plt.title(title)

pre_x = pre_two_d_embeddings[:, 0]
pre_y = pre_two_d_embeddings[:, 1]

post_x = post_two_d_embeddings[:, 0]
post_y = post_two_d_embeddings[:, 1]

plt.figure(figsize=(10, 20))  # in inches

plt.subplot(5,2,1)
plot(pre_two_d_embeddings[:, 0], pre_two_d_embeddings[:, 1], labels_gc, 'Pre GC-Ratio')

plt.subplot(5,2,2)
plot(post_two_d_embeddings[:, 0], post_two_d_embeddings[:, 1], labels_gc, 'Post GC-Ratio')

plt.subplot(5,2,3)
plot(pre_two_d_embeddings[:, 0], pre_two_d_embeddings[:, 1], labels_entropy, 'Pre Entropy')

plt.subplot(5,2,4)
plot(post_two_d_embeddings[:, 0], post_two_d_embeddings[:, 1], labels_entropy, 'Post Entropy')

plt.subplot(5,2,5)
plot(pre_two_d_embeddings[:, 0], pre_two_d_embeddings[:, 1], labels_frequence, 'Pre Frequence (log10)')

plt.subplot(5,2,6)
plot(post_two_d_embeddings[:, 0], post_two_d_embeddings[:, 1], labels_frequence, 'Post Frequence (log10)')

plt.subplot(5,2,7)
plot(pre_two_d_embeddings[:, 0], pre_two_d_embeddings[:, 1], labels_frequence_line, 'Pre LINE Frequence (log10)')

plt.subplot(5,2,8)
plot(post_two_d_embeddings[:, 0], post_two_d_embeddings[:, 1], labels_frequence_line, 'Post LINE Frequence (log10)')

plt.subplot(5,2,9)
plot(pre_two_d_embeddings[:, 0], pre_two_d_embeddings[:, 1], labels_frequence_sine, 'Pre SINE Frequence (log10)')

plt.subplot(5,2,10)
plot(post_two_d_embeddings[:, 0], post_two_d_embeddings[:, 1], labels_frequence_sine, 'Post SINE Frequence (log10)')


plt.show()

### 3D

In [ ]:
tsne_3d = TSNE(perplexity=50, n_components=3, init='pca', n_iter=5000)

#post_3d_embeddings = tsne_3d.fit_transform([post_normalized_embeddings[s, :] for s in random_kmers])

# All
post_3d_embeddings = tsne_3d.fit_transform(post_normalized_embeddings)

In [ ]:
post_x = post_3d_embeddings[:, 0]
post_y = post_3d_embeddings[:, 1]
post_z = post_3d_embeddings[:, 2]

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection='3d')
plt.gca().patch.set_facecolor('white')

label_type = labels_frequence

norm = colors.Normalize(vmin=min(label_type), vmax=max(label_type))
ax.scatter(post_x, post_y, post_z, alpha=0.3, c=label_type, cmap=plt.get_cmap('jet'), norm=norm)

plt.show()